In [1]:
import re
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import codecs
from collections import defaultdict
import time
import operator
path = 'C:/Users/OptimusPrime/Desktop/Studia/NLP1/Natural-Language-Processing-1/Week 7/Data'

In [6]:
voc = dict()
with codecs.open(path + '\\odm.txt', 'r', 'utf-8') as f:
    text = f.read().lower() 
    lines = text.split('\r\n') 
    for line in lines:
        words = np.array(line.split(', '))
        for word in words:
            voc[word] = words[0]

In [11]:
authors = ['Szklarski', 'Dukaj', 'MacLean', 'Norton', 'McCaffrey', 'Bulyczow', 'Scott', 'Gibson']
texts_with_authors = defaultdict(list)
for file in os.listdir(path + '/Texts/'): 
    for author in authors:                         
        if author in file:
            print(file)
            with codecs.open(path + '/Texts/' + file, 'r', 'utf-8') as f:
                text = f.read().lower()
                preprocessed_text = re.sub(r'#\d+', '##', text)
                preprocessed_text = re.sub(r'\n', ' ', preprocessed_text)
                preprocessed_text = re.sub(r'[§$#¨|\^\\,.\-\'\"()?:;+\/\d+&`!\[\]@<>%*~{}=\x96\x91\x84\x8c\x93\x9c\x9f\x92\x80\x94\x95°\x97]', '', preprocessed_text)
                preprocessed_text = re.sub(r'   ', '  ', preprocessed_text)
                preprocessed_text = re.sub(r'  ', ' ', preprocessed_text)
                texts_with_authors[author].append(preprocessed_text)  

Alfred_Szklarski_-_Tomek_u_zrodel_Amazonki.txt
Alfred_Szklarski_-_Tomek_wsrod_lowcow_glow.txt
Alfred_Szklarski_-_Tomek_w_Gran_Chaco.txt
Alfred_Szklarski_-_Tomek_w_Krainie_Kangurow.txt
Alistair_MacLean_-_Athabaska.txt
Alistair_MacLean_-_HMS_Ulisses.txt
Alistair_MacLean_-_Mroczny_Krzyzowiec.txt
Alistair_MacLean_-_Przelecz_zlamanego_serca.txt
Alistair_MacLean_-_Stacja_arktyczna_Zebra.txt
Alistar_MacLean_-_Zlote_rendez-vous.txt
Andre_Norton_-_Brama_Kota.txt
Andre_Norton_-_Bunt_Agentow.txt
Andre_Norton_-_Cesarska_corka.txt
Andre_Norton_-_Czarodziej_ze__wiata_Czarownic.txt
Andre_Norton_-_Czary_Swiata_Czarownic.txt
Andre_Norton_-_Galaktyczni_rozbitkowie.txt
Andre_Norton_-_Garan_niesmiertelny.txt
Andre_Norton_-_Gniazdo_Gryfa.txt
Anne_McCaffrey_-_Krysztalowa_Wiez.txt
Anne_McCaffrey_-_Mistrz_harfiarzy_z_Pern.txt
Anne_McCaffrey_-_Niebiosa_Pern.txt
Anne_McCaffrey_-_Ocaleni_-_Planeta_dinozaurow_2.txt
Anne_McCaffrey_-_Opowiesci_Nerilki.txt
Bulyczow_Kiryl_-_Ludzie_jak_ludzie_SCAN-dal_756.txt
Bulyczow

In [12]:
def to_basic_form(matchobj):
    word = matchobj.group(0)
    if word in voc:
        return voc[word]
    else:
        return None


word_pattern = re.compile('\w+')
texts_with_authors_basic_forms = defaultdict(list)
for author in texts_with_authors:
    for text in texts_with_authors[author]:
        texts_with_authors_basic_forms[author].append(word_pattern.sub(to_basic_form, text))

In [13]:
all_texts = list()
for key in texts_with_authors_basic_forms:
    all_texts += texts_with_authors_basic_forms[key]

cv = CountVectorizer(token_pattern='(?u)\\b\\w+\\b')

cv_freq_matrix = cv.fit_transform(all_texts)
distinct_words = cv.get_feature_names()

number_of_distinct_words = cv_freq_matrix.shape[1]

words_indices = dict(zip(distinct_words, range(len(distinct_words))))

In [14]:
markov_models = defaultdict(lambda: defaultdict(int))
for author in authors:    
    concatenated_texts = []
    for text in texts_with_authors_basic_forms[author]:
        concatenated_texts += text.split()
    print('Length in words of ' + author + ' texts: ' + str(len(concatenated_texts)))
    
    words_counts = defaultdict(int)
    for word in concatenated_texts:
        words_counts[word] += 1  
        
    length_of_concatenated_texts = len(concatenated_texts)
    for i in range(length_of_concatenated_texts):
        if i == length_of_concatenated_texts - 1:
            break;  
        markov_models[author][(words_indices[concatenated_texts[i]], words_indices[concatenated_texts[i+1]])] = (1 / words_counts[concatenated_texts[i]])

Length in words of Szklarski texts: 257222
Length in words of Dukaj texts: 207116
Length in words of MacLean texts: 369835
Length in words of Norton texts: 400825
Length in words of McCaffrey texts: 382604
Length in words of Bulyczow texts: 128606
Length in words of Scott texts: 395145
Length in words of Gibson texts: 295350


In [15]:
all_scores = defaultdict(list)
for real_author in texts_with_authors_basic_forms:
    books_list = list()
    for i in range(len(texts_with_authors_basic_forms[real_author])):
        chosen_book = texts_with_authors_basic_forms[real_author][i].split()

        words_counts = defaultdict(int)
        for word in chosen_book:
            words_counts[word] += 1  

        chosen_book_markov_model = defaultdict(int)
        length_of_chosen_book = len(chosen_book)
        for i in range(length_of_chosen_book):
            if i == length_of_chosen_book - 1:
                break;  
            chosen_book_markov_model[(words_indices[chosen_book[i]], words_indices[chosen_book[i+1]])] = (1 / words_counts[chosen_book[i]])
        
        scores = defaultdict(int) 
        for author in markov_models:
            indices_to_compare = set(markov_models[author]) | set(chosen_book_markov_model)
            for ind in indices_to_compare:                   
                scores[author] += abs(chosen_book_markov_model[ind] - markov_models[author][ind])
        books_list.append(scores)
        print("Chosen book's real author: " + real_author)
        print("Differences in writing style:")
        
        for a in scores:
            print(a + ' <- ' + str(scores[a]))
        print('')
    all_scores[real_author] = books_list

Chosen book's real author: Szklarski
Differences in writing style:
Szklarski <- 12525.111551230862
Dukaj <- 20828.09968495682
MacLean <- 20150.116528984843
Norton <- 18989.163822122122
McCaffrey <- 20270.66744796892
Bulyczow <- 16667.14526233993
Scott <- 18918.5043444194
Gibson <- 20540.14459807793

Chosen book's real author: Szklarski
Differences in writing style:
Szklarski <- 12480.445937143626
Dukaj <- 20632.349978432132
MacLean <- 19953.627525420874
Norton <- 18817.128812555868
McCaffrey <- 20060.99082288223
Bulyczow <- 16470.768232937633
Scott <- 18745.510207079795
Gibson <- 20326.985727603813

Chosen book's real author: Szklarski
Differences in writing style:
Szklarski <- 12145.805428400545
Dukaj <- 21322.835976466024
MacLean <- 20609.777245417627
Norton <- 19474.18365135978
McCaffrey <- 20745.201560441372
Bulyczow <- 17164.027174108745
Scott <- 19420.212992748642
Gibson <- 21016.17309306723

Chosen book's real author: Szklarski
Differences in writing style:
Szklarski <- 12297.23

Chosen book's real author: Bulyczow
Differences in writing style:
Szklarski <- 12830.818655302492
Dukaj <- 15493.307229616095
MacLean <- 14792.538735985645
Norton <- 13653.977206028523
McCaffrey <- 14914.581645398968
Bulyczow <- 10901.550326977016
Scott <- 13577.651536635552
Gibson <- 15119.011688388224

Chosen book's real author: Bulyczow
Differences in writing style:
Szklarski <- 13458.729908713714
Dukaj <- 16109.671827060121
MacLean <- 15422.65452628445
Norton <- 14279.8950600762
McCaffrey <- 15553.018613494072
Bulyczow <- 11048.913860115565
Scott <- 14211.150809179766
Gibson <- 15748.83862926487

Chosen book's real author: Bulyczow
Differences in writing style:
Szklarski <- 13010.988325175047
Dukaj <- 15656.484231208746
MacLean <- 14967.736777207372
Norton <- 13839.489413890655
McCaffrey <- 15099.85628498277
Bulyczow <- 10887.40328547799
Scott <- 13756.969001866859
Gibson <- 15301.31835546898

Chosen book's real author: Bulyczow
Differences in writing style:
Szklarski <- 12939.6095

In [16]:
good = 0
alls = 0
for ra in all_scores:
    good = 0
    alls = 0
    for book in all_scores[ra]:        
        pred_author = min(book.items(), key=operator.itemgetter(1))[0]
        alls += 1
        if pred_author == ra:
            good += 1   
    print('Accuracy for ' + ra + ' books: ' + str(good/alls))

Accuracy for Szklarski books: 1.0
Accuracy for MacLean books: 0.8333333333333334
Accuracy for Norton books: 0.75
Accuracy for McCaffrey books: 0.8
Accuracy for Bulyczow books: 1.0
Accuracy for Scott books: 0.8333333333333334
Accuracy for Dukaj books: 1.0
Accuracy for Gibson books: 1.0
